In [1]:
# commande utilisée pour générer le PGN avec pgn-extract :
# ./pgn-extract -ogamesShortMoy.pgn -Wlalg --stopafter 100 --nomovenumbers --noresults --notags -M db_1.pgn
# https://www.cs.kent.ac.uk/people/staff/djb/pgn-extract/

# on parse le fichier pgn (séparation des différentes parties)

import itertools
import numpy as np

listeTotale=[]
with open("gamesShortShort.pgn") as f: # ouverture d'un fichier PGN
    nGame=0
    for line in f:
        l = line.split(' ')
        for k in range(0,len(l)):
            if l[k].endswith('\n'):
                l[k] = l[k].split('\n', 1)[0]
        listeTotale = listeTotale + l
print(listeTotale[0:10]) # on imprime un exemple


['d2d4', '{', 'book', '}', 'g8f6', '{', 'book', '}', 'c2c4', '{']


In [2]:
# on nettoie les données obtenues

import itertools
import numpy as np

tab = np.load('gamesPGNBIG.npy') # on ouvre en fait le tableau précédent. ici il est stocké dans un fichier
                                 # car le code précédent a été exécuté sur une machine de calcul haute performance
arrayX = []
for x in tab:
    # à paral.
    arrayY = []
    for k in range(0,len(x)):
        if x[k].endswith('\n'):
            x[k] = x[k].split('\n', 1)[0]
        if '/' in x[k]:
            x[k] = x[k].split('/', 1)[0]
        if x[k].endswith('+'):
            x[k] = x[k].split('+', 1)[0]
        if x[k].startswith('+'):
            x[k] = x[k].split('+', 1)[1]
        if x[k].endswith('#'):
            x[k] = x[k].split('#', 1)[0]
        if 'M' in x[k]:
            if '-' in x[k]:
                x[k] = '-100'
            else:
                x[k] = '100'
        if x[k] != '{' and x[k] != '}' and x[k] != '' and not(x[k].endswith('s')) and not(x[k].endswith('s,')): # enlève aussi 'mates'
            arrayY.append(x[k])
    arrayX.append(arrayY)

In [4]:
# on génère deux jeux de données propres : les parties et leurs évaluations

gamesCoups = []
gamesLabels = []
for i in range(0,len(arrayX)):
    coups = []
    evaluations = []
    for j in range(0,len(arrayX[i])):
        if j % 2 == 0:
            coups.append(arrayX[i][j])
        if j % 2 == 1:
            if arrayX[i][j] == 'book':
                arrayX[i][j] = '0'
            evaluations.append(float(arrayX[i][j]))
    gamesCoups.append(coups)
    gamesLabels.append(evaluations)
print(gamesCoups[0])
print(gamesLabels[0])

['d2d4', 'g8f6', 'c2c4', 'e7e6', 'g1f3', 'd7d5', 'b1c3', 'd5c4', 'd1a4', 'c7c6', 'a4c4', 'b7b5', 'c4b3', 'b8d7', 'c1g5', 'd8a5', 'e2e3', 'a7a6', 'f1e2', 'c6c5', 'e1g1', 'c8b7', 'f1d1', 'h7h6', 'g5f4', 'f8e7', 'a2a3', 'a8c8', 'f3e5', 'd7e5', 'd4e5', 'f6d7', 'b3c2', 'a5c7', 'c3e4', 'b7e4', 'c2e4', 'c5c4', 'a1b1', 'd7c5', 'e4f3', 'e8g8', 'f3g3', 'g8h8', 'g3f3', 'f8d8', 'd1d4', 'c5b3', 'd4d8', 'c8d8', 'f3h5', 'h8g8', 'h5g4', 'g8h8', 'a3a4', 'c7a5', 'g4h5', 'h8g8', 'h5g4', 'g8f8', 'a4b5', 'a6b5', 'f4g3', 'a5a2', 'b1d1', 'd8d1', 'e2d1', 'a2b2', 'f2f4', 'e7c5', 'g3f2', 'c4c3', 'g4f3', 'f8g8', 'f2e1', 'c3c2', 'f3a8', 'c5f8', 'd1c2', 'b2c2', 'e1b4', 'b3c5', 'b4c5', 'c2c5', 'a8e4', 'c5c1', 'g1f2', 'c1d2', 'f2f3', 'f8c5', 'h2h4', 'h6h5', 'g2g4', 'd2d1', 'f3f2', 'd1g4', 'e4a8', 'g8h7', 'a8e4', 'h7h6', 'e4a8', 'g4f4', 'f2e1', 'f4b4', 'e1f2', 'b4d2', 'f2f1', 'd2e3', 'a8a2', 'e3g1', 'f1e2', 'g1f2', 'e2d3', 'f2a2', 'd3c3', 'c5e3', 'c3b4', 'a2c4', 'b4a3', 'e3c1']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [5]:
# comme les évaluations sont calculées pour les deux joueurs depuis leurs points de vue, on fait en sorte
# de tout voir du point de vue des blancs

for i in range(0,len(gamesLabels)):
    for j in range(0, len(gamesLabels[i])):
        if j % 2 == 1 and gamesLabels[i][j] != 0.0:
            gamesLabels[i][j] = -gamesLabels[i][j]


In [6]:
# on enregistre enfin les beaux fichiers de données dans des tableaux Numpy

gamesCoupsNumpy = np.asarray(gamesCoups)
np.save('gamesCoups.npy', gamesCoupsNumpy)

rawLabels = []

for i in range(0,len(gamesLabels)):
    for j in range(0, len(gamesLabels[i])):
        rawLabels.append(gamesLabels[i][j])

rawLabelsNumpy = np.asarray(rawLabels)
np.save('rawLabels.npy', rawLabelsNumpy)

In [7]:
print(len(gamesCoupsNumpy))

30001
